In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()

In [ ]:
data = pd.read_excel(r'C:\Users\nijat\Desktop\PYTHON - KURS\Ders 21\Daily_Demand_Forecasting_Orders_reg.xls')
data.head(5)

In [ ]:
data.describe(include = 'all')

In [ ]:
data.dtypes

In [ ]:
data.isnull().sum()

In [ ]:
data.corr()

In [ ]:
data.columns

In [ ]:
data.head()

In [ ]:
data = data[['Day of the week', 'Order type B', 'Banking orders (1)', 'Target (Total orders)']]
data.head()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

variables = data[['Day of the week', 'Order type B', 'Banking orders (1)']]
vif = pd.DataFrame()

vif["VIF"] = [variance_inflation_factor(variables.values, i) for i in range(variables.shape[1])]
vif["Features"] = variables.columns
vif

In [ ]:
data = data[['Day of the week', 'Order type B', 'Banking orders (1)', 'Target (Total orders)']]
data.head()

In [ ]:
for i in data[['Day of the week', 'Order type B', 'Banking orders (1)']]:
    sns.boxplot(data=data,x=data[i])
    plt.show()

In [ ]:
q1=data.quantile(0.25)
q3=data.quantile(0.75)
IQR=q3-q1

Lower = q1-IQR
Upper = q3+IQR

In [ ]:
for i in data[['Day of the week', 'Order type B', 'Banking orders (1)']]:
    data[i] = np.where(data[i] > Upper[i],Upper[i],data[i])
    data[i] = np.where(data[i] < Lower[i],Lower[i],data[i])
    sns.boxplot(data=data,x=data[i])
    plt.show()

In [ ]:
data.columns

In [ ]:
y = data[['Target (Total orders)']]

X= data[['Day of the week', 'Order type B', 'Banking orders (1)']]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
x_scaled = scaler.transform(X)

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn import metrics

def evaluate(model, x_test, y_test):
    y_pred = model.predict(x_test)
    mae = metrics.mean_absolute_error(y_test, y_pred)
    mse = metrics.mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = metrics.r2_score(y_test, y_pred)
    print('Model Performance')
    print('Accuracy is', r2*100, '%')
    
    return mae
    return mse
    return rmse
    return r2

In [ ]:
from sklearn.svm import SVR

base_model = SVR(kernel = 'linear')
base_model.fit(x_train, y_train)
base_accuracy = evaluate(base_model, x_test, y_test)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

kernel = ['linear', 'poly']
# kernel trick

gamma = ['scale', 'auto'] 
#Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’.

#if gamma='scale' (default) is passed then it uses 1 / (n_features * X.var()) as value of gamma
#if ‘auto’, uses 1 / n_features.

C = [1, 10, 100, 1e3, 1e4, 1e5, 1e6]
# Must be strictly positive.

epsilon = [0.1 , 0.01, 0.001, 0.0001]


# Create the random grid
random_grid = {'kernel': kernel,
               'gamma': gamma,
               'C': C,
               'epsilon': epsilon}
print(random_grid)

In [ ]:
svr_random = RandomizedSearchCV(estimator = base_model, param_distributions = random_grid, n_iter = 1, cv = 3, verbose=1, n_jobs = -1)
# Fit the random search model
svr_random.fit(x_train, y_train)

In [ ]:
svr_random.best_params_

In [ ]:
optimized_model = svr_random.best_estimator_
optmized_accuracy = evaluate(optimized_model, x_test, y_test)